In [1]:
import cv2
import os
import time
from memory_queues import square_queue, gui_queue
from memory_logic import register_card
from sift_utils import extract_sift_signature, auto_crop_inside_white_edges, draw_oriented_bounding_box
from recorded_positions import pick_positions, drop_positions, home_pose, scan_pose
from pyniryo2 import NiryoRos, NiryoRobot, Vision
import pyniryo
from matplotlib import pyplot as plt
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
robot_ip_address = "169.254.200.200"
robot = NiryoRobot(robot_ip_address)
robot.arm.calibrate_auto()
# robot.activate_vacuum()
robot.tool.release_with_tool()
robot.arm.move_pose(home_pose)


In [ ]:
COLOR=[0,0,255]
robot.led_ring.flash(COLOR, period=0.35, iterations=5, wait=True) 
robot.led_ring.turn_off()

In [ ]:
robot.arm.get_arm_max_velocity()
robot.arm.move_pose([0.26, 0.03, 0.17, 1.55, 0.2, 1.497])



In [ ]:
pose_read = robot.arm.get_pose()
print(f'{round(pose_read.x,2), round(pose_read.y,2), round(pose_read.z,2), round(pose_read.roll,2), round(pose_read.pitch,2), round(pose_read.yaw,4)} ')

In [ ]:
pose_read = robot.arm.get_pose()
print(f'{round(pose_read.x,2), round(pose_read.y,2), round(pose_read.z,2), round(pose_read.roll,2), round(pose_read.pitch,2), round(pose_read.yaw,4)} ')

In [ ]:
#L2
robot.arm.move_pose([0.226, 0.226, 0.07, -2.10, 1.48, -2.1716])
#robot.arm.move_pose([0.226, 0.226, 0.07, -2.28, 1.48, -2.1716])
#robot.tool.grasp_with_tool()
#robot.arm.move_pose([0.15, 0.0, 0.055, 1.48, 1.5, 1.5422])
#robot.tool.grasp_with_tool()

In [ ]:
robot.arm.move_pose([0.230, -0.22, 0.07, 2.69, 1.45, 2.862])


In [ ]:
#R2
robot.arm.move_pose([0.147, 0.074, 0.07, -2.85, 1.52, -2.5403])
robot.arm.move_pose([0.147, 0.074, 0.053, -2.55, 1.52, -2.5403])
robot.tool.grasp_with_tool()
robot.arm.move_pose([0.147, 0.074, 0.07, -2.55, 1.52, -2.5403])
robot.arm.move_pose([0.230, -0.22, 0.07, 2.61, 1.47, 2.8617])
robot.tool.release_with_tool()




In [ ]:
robot.arm.move_pose([0.22, -0.0, 0.21, -2.96, 1.54, -2.9783]) #home
robot.arm.move_pose([0.15, 0.14, 0.056, 3.0, 1.48, 3.0651]) #A1 drop pose
robot.arm.move_pose([0.15, 0.14, 0.05, 3.0, 1.48, 3.0651]) #A1 pick pose
robot.tool.grasp_with_tool()
robot.arm.move_pose([0.15, 0.14, 0.056, 3.0, 1.48, 3.0651]) #A1 drop pose
robot.arm.move_pose([0.228, -0.22, 0.07, 2.67, 1.47, 2.8617]) #L1 stack
robot.tool.release_with_tool()


In [ ]:
robot.arm.move_pose([0.22, -0.0, 0.21, -2.96, 1.54, -2.9783]) #home
robot.arm.move_pose([0.147, 0.074, 0.07, -2.95, 1.52, -2.5403]) #A1 drop pose
robot.arm.move_pose([0.147, 0.074, 0.055, -2.95, 1.52, -2.5403]) #A1 pick pose
robot.tool.grasp_with_tool()
robot.arm.move_pose([0.147, 0.074, 0.07, -2.95, 1.52, -2.5403]) #A1 drop pose
robot.arm.move_pose([0.228, -0.22, 0.07, -2.95, 1.47, 2.8617]) #L1 stack
robot.tool.release_with_tool()


In [ ]:
robot.arm.move_pose([0.228, -0.22, 0.07, 2.69, 1.47, 2.8617]) #L1 stack


In [ ]:
#scan pose
robot.arm.move_pose([0.22, -0.0, 0.21, -2.96, 1.54, -2.9783]) #home
robot.arm.move_pose([0.37, 0.15, 0.05, 2.50, 1.55, 2.4585])
robot.tool.grasp_with_tool()
robot.arm.move_pose([0.37, 0.15, 0.065, 2.50, 1.55, 2.4585])
robot.arm.move_pose([0.26, 0.03, 0.17, 1.55, 0.2, 1.497])#scan
robot.wait(5)
robot.arm.move_pose([0.22, -0.0, 0.21, -2.96, 1.54, -2.9783])#home
robot.arm.move_pose([0.37, 0.15, 0.05, 2.50, 1.55, 2.4585])
robot.tool.release_with_tool()
robot.arm.move_pose([0.22, -0.0, 0.21, -2.96, 1.54, -2.9783])#home







In [ ]:
robot.arm.move_pose([0.22, -0.0, 0.21, -2.96, 1.54, -2.9783]) #home
robot.arm.move_pose([0.37, 0.15, 0.05, 2.50, 1.55, 2.4585]) #A1 pick pose
robot.tool.grasp_with_tool()
robot.arm.move_pose([0.37, 0.15, 0.065, 2.50, 1.55, 2.4585]) #A1 drop pose
robot.arm.move_pose([0.3, 0.225, 0.06, -2.65, 1.52, -2.6712]) #L1 stack
robot.tool.release_with_tool()


In [ ]:
pose_read = robot.arm.get_pose()
print(f'{round(pose_read.x,2), round(pose_read.y,2), round(pose_read.z,2), round(pose_read.roll,2), round(pose_read.pitch,2), round(pose_read.yaw,4)} ')

In [ ]:
robot.arm.move_pose([0.26, 0.03, 0.15, 1.55, 0.2, 1.497])


In [ ]:
robot.tool.grasp_with_tool()

In [ ]:
robot.tool.release_with_tool()


In [ ]:
robot.tool.release_with_tool()
robot.arm.move_pose([0.22, 0.22, 0.07, -2.28, 1.48, -2.1716])
robot.tool.grasp_with_tool()
robot.arm.move_pose([0.3, 0.15, 0.065, -2.85, 1.46, -2.8733])
robot.tool.release_with_tool()
robot.arm.move_pose([0.3, 0.15, 0.05, -2.85, 1.46, -2.8733])
robot.tool.grasp_with_tool()
robot.arm.move_pose([0.3, 0.15, 0.065, -2.85, 1.46, -2.8733])
robot.arm.move_pose([0.22, 0.22, 0.07, -2.28, 1.48, -2.1716])
robot.tool.release_with_tool()




In [ ]:

robot.arm.move_pose([0.228, -0.22, 0.07, 2.97, 1.47, 2.8617])


In [ ]:
import cv2
import pyniryo
from pyniryo2 import NiryoRos, Vision
import numpy as np

CARD_BOX = (270, 190, 190, 190)  # x,y,h,w

ros_instance = NiryoRos("169.254.200.200")
vision = Vision(ros_instance)

print("[SCAN] Looking for card at ")
last_center = stable_since = detection_time = None
while True:
    img_compressed = vision.get_img_compressed()
    
    # Check if the image was successfully captured
    if img_compressed is None:
        print("[ERROR] Could not get compressed image from the robot.")
        continue  # Skip the rest of the loop and try again

    img_uncompressed = pyniryo.uncompress_image(img_compressed)
    
    # Add a check for the uncompressed image as well
    if img_uncompressed is None:
        print("[ERROR] Failed to uncompress image.")
        continue

    camera_info = vision.get_camera_intrinsics()
    img = pyniryo.undistort_image(img_uncompressed, camera_info.intrinsics, camera_info.distortion)

    frame = cv2.resize(img, (640, 480))
    cv2.rectangle(frame, (CARD_BOX[0], CARD_BOX[1]), (CARD_BOX[0] + CARD_BOX[2], CARD_BOX[1] + CARD_BOX[3]), (0, 255, 0), 2)
    cv2.imshow("Adjust ROI", frame)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cv2.destroyAllWindows()